## 不常用的语法
- [for...else](#for_else)
- [assert断言](#assert)
- [try-except-else-finally](#try_except_else_finally)
- [str.translate](#str_translate)
    - [str.translate vs str.replace](#replace_vs_translate)
- [function.__annotations__](#function_annotations)
- [序列比较](#sequence)

### <a id='for_else'>for...else...</a>
for完全循环（没有break时）执行else里的代码块      
应用：在else里删除一些“哨兵”变量，如果中断了这些变量将被保留，便于后续跟踪     

In [26]:
for i in range(10):
    pass 
else:
    print('Done')

Done


In [29]:
for i in range(10):
    if i==1:
        print('break')
        break
else:
    print('Done')

break


### <a id='assert'>Assert断言语句<a>
```
语法：     
    assert 语句   
    assert 语句, 异常信息       
当语句为真时，无输出
当语句为假时，抛出AssertionError异常及异常信息（如果有）       
异常信息可以是任意数据类型
```
注意：当且仅当解释器是非优化条件运行时（默认非优化，__debug__: True）assert语句才会被执行     
所以不要将assert语句用于数据检查尤其是安全检查！       

In [1]:
__debug__

True

In [6]:
__debug__ = False   #__debug__实际是一个值，无法再被赋值

SyntaxError: assignment to keyword (<ipython-input-6-ac4675a98060>, line 1)

In [2]:
assert 2==2   #断言为真时，无输出

In [25]:
assert 0   #断言之后的语句为假时抛出AssertionError异常

AssertionError: 

In [11]:
assert not 1

AssertionError: 

In [19]:
assert 2==1, 'the 2 numbers are not the same' #定义异常信息

AssertionError: the 2 numbers are not the same

In [21]:
assert 2==1, ('good', 'bad')

AssertionError: ('good', 'bad')

In [22]:
assert 2==1, 9999

AssertionError: 9999

In [24]:
assert 2==1, {'hi':89}

AssertionError: {'hi': 89}

#### 注意避免被断言的语句写成非空tuple
```
> 如何避免？
1、pyflask 检查脚本： $ pyflask xxx.py             
2、egrep 检查脚本：$ egrep 'assert *\(' xxx.py
```

In [10]:
assert (not 1,)   #当语句被括号包裹，且带一个逗号时，其实已经是一个非空tuple了

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-10-c12a0a73bc12>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (not 1,)


#### 应用：处理/捕获“不易复现”的场景，用assert而不是自定义异常的好处是在生产环境可以略过不处理     
```
if condition1:
    do_planA()
elif condition2:
    do_planB()
else unexpected:
    assert 0, (write some message)  #抛出断言异常
```

<img src='./imgs/assert_without_execution.png'>

### <a id='try_except_else_finally'>try-except-else-finally</a>
```
'try-statement'
	try:
		try_suite
	except Exception1:
		handle_for_exception
	except (Exception2, Exception3) as reason:   #python2：except Exception, reason
		handle_for_exception_2_and_3_plus_reason
	except:
		handle_for_other_exceptions
	else:
		no_exceptions_detected_suite
	finally:
		always_excute_suite
```

In [39]:
'基本捕获'
try:
    raise ValueError('haha')
except Exception as e:
    print('catched:',type(e), str(e))

catched: <class 'ValueError'> haha


In [38]:
'else代码块: try代码块无异常时执行'
try:
    pass
except Exception as e:
    print('catched:',type(e), str(e))
else:        
    print('safed')

safed


In [42]:
'finally代码块: 最终必执行'
try:
    pass
except Exception as e:
    print('catched:',type(e), str(e))
else:        
    print('safed')
finally:
    print('end')

safed
end


In [41]:
'finally代码块: 最终必执行，有异常时也执行'
try:
    raise ValueError('haha')
except Exception as e:
    print('catched:',type(e), str(e))
finally:
    print('end')

catched: <class 'ValueError'> haha
end


In [92]:
'finally + raise: 先执行finally的代码块，再raise'
try:
    raise ValueError('haha')
except Exception as e:
    print('catched:',type(e), str(e))
    raise e   # 在finally代码块执行完后执行
finally:
    print('end')

catched: <class 'ValueError'> haha
end


ValueError: haha

### <a id='str_translate'>str.translate</a>

In [65]:
s = 'hi9Megan8'

In [73]:
help(s.translate)  # -> str, 字符映射替换，如果映射为None则删除该字符

Help on built-in function translate:

translate(...) method of builtins.str instance
    S.translate(table) -> str
    
    Return a copy of the string S in which each character has been mapped
    through the given translation table. The table must implement
    lookup/indexing via __getitem__, for instance a dictionary or list,
    mapping Unicode ordinals to Unicode ordinals, strings, or None. If
    this operation raises LookupError, the character is left untouched.
    Characters mapped to None are deleted.



In [69]:
help(str.maketrans)   # -> dict: {unicode(character):unicode(character), ...}, 制作映射表，返回作为translate的参数

Help on built-in function maketrans:

maketrans(x, y=None, z=None, /)
    Return a translation table usable for str.translate().
    
    If there is only one argument, it must be a dictionary mapping Unicode
    ordinals (integers) or characters to Unicode ordinals, strings or None.
    Character keys will be then converted to ordinals.
    If there are two arguments, they must be strings of equal length, and
    in the resulting dictionary, each character in x will be mapped to the
    character at the same position in y. If there is a third argument, it
    must be a string, whose characters will be mapped to None in the result.



In [55]:
dict.fromkeys('234')

{'2': None, '3': None, '4': None}

In [60]:
str.maketrans(dict.fromkeys('234'))  #只有一个参数时，x需是一个dict

{50: None, 51: None, 52: None}

In [71]:
str.maketrans('123','zzz')   #参数x、y非空，要求字符串长度要一致

{49: 122, 50: 122, 51: 122}

In [72]:
str.maketrans('123','zz')   #参数x、y非空，要求字符串长度要一致

ValueError: the first two maketrans arguments must have equal length

In [64]:
type(str.maketrans('123','zzz'))

dict

In [54]:
s.translate(str.maketrans('0123456789', ' '*10))  #将数字都换成空格，注意maketrans的2个参数长度要一致

'hi Megan '

In [59]:
s.translate(str.maketrans(dict.fromkeys('0123456798')))   #删除指定字符

'hiMegan'

In [75]:
help(str.replace)   #子字符串替换

Help on method_descriptor:

replace(...)
    S.replace(old, new[, count]) -> str
    
    Return a copy of S with all occurrences of substring
    old replaced by new.  If the optional argument count is
    given, only the first count occurrences are replaced.



####  <a id='replace_vs_translate'>str.replace vs str.translate</a>
前者偏向于对有秩序的子序列的替换，后者不讲究秩序

> 无序替换: 删除数字

In [81]:
ss = 'Hi123This456is789Jone0!'

In [82]:
ss.replace('9','').replace('8','').replace('7','').replace('6','').replace('5','').replace('4','').replace('3','').replace('2','').replace('1','').replace('0','')

'HiThisisJone!'

In [83]:
ss.translate(str.maketrans(dict.fromkeys('0123456789')))

'HiThisisJone!'

>有序替换：替换子字符串

In [84]:
sss = 'Good morning, Megan!'

In [85]:
sss.replace('Megan', 'Jo')

'Good morning, Jo!'

In [86]:
#sss.translate(?) 做不到

### <a id='function_annotations'>function.__annotations__</a>
函数注解

In [88]:
def func(a:str, b:int) -> str:
    pass

In [89]:
func.__annotations__

{'a': str, 'b': int, 'return': str}

In [91]:
func(23,34)

### <a id='compare'>数据比较</a>
相同数据类型可以比较，不同数据类型比较一般会报错

In [3]:
(1, 2, 3) < (1,3, 4)

True

In [4]:
(1, 2, 3) < [1,3, 4]  

TypeError: '<' not supported between instances of 'tuple' and 'list'

In [5]:
[1, 2, 3] < {1,3, 4}  

TypeError: '<' not supported between instances of 'list' and 'set'

> set之间的比较：可以比较是否相同（个数，所含元素），但无法比较谁大谁小

In [6]:
{1,2,3} > {2, 3, 4}

False

In [35]:
{3,2,4} == {2, 3, 4}

True

In [34]:
{3,2,4} >= {2, 3, 4}

True

In [21]:
{3,2,1} >= {1,2,4}

False

In [20]:
{3,2,1} <= {1,2,4}

False

In [22]:
[1,2,3] == {1,2,3}

False

> 字典之间也只支持相等比较，非相等比较会报错

In [26]:
d1={1:23}

In [27]:
d2={2:24}

In [33]:
d1>=d2

TypeError: '>=' not supported between instances of 'dict' and 'dict'

In [29]:
d1==d2

False

In [31]:
d1={1:23,2:24}
d2={2:24, 1:23}

In [32]:
d1==d2

True